<h2>Final Project</h2>
    <p>Lego Program - v0.1.1</p>
    <p>Rashan Walker</p>

In [1]:
import csv
import glob
import gzip
import json
import os
import pandas
import re
import requests
import shutil
import sqlite3
import zipfile
from bs4 import BeautifulSoup

data_folder = './data/'
brickset_api_root = 'https://brickset.com/api/v3.asmx/'

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) ' \
           'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [3]:
def close_DB_Resources(dbconn):
    '''
    Purpose: Close a database connection
    Input: Database connection
    Default(s): NONE
    Output: NONE
    Notes: (blank)

    Credit == Github: lan33-ccac
    '''
    try:
        dbconn.close()
        print()
        print('DB resources were closed successfully.')
    except sqlite3.Error as error:
        print('Error occurred closing DB resources.', error)

In [4]:
def remove_file(file):
    '''
    Purpose: Delete a file
    Input: File path
    Default(s): NONE
    Output: NONE
    Notes: (blank)
    '''
    try:
        os.remove(file)
    except Exception as error:
        print("Error in function: remove_file()")
        print(error)

In [5]:
def show_files(top = '.'):
    '''
    Purpose: Show files in a folder
    Input: System Path
    Default(s): '.'
    Output: NONE
    Notes: (blank)
    '''
    try:
        for loc, dirs, files in os.walk(top):
            if loc == '.':
                print('***********************************************************')
                print('Files(s):', ', '.join(files))
                print('***********************************************************')
    except Exception as error:
        print("Error in function: show_files()")
        print(error)

In [6]:
def get_file(url, file_type='.csv.gz'):
    '''
    Purpose: Download a file
    Input: File Name and URL of file
    Default(s): File type of .csv.gz
    Output: (None)
    Notes: (Blank)
    '''
    path = './temp/'
    search = re.compile(r'\w+'+file_type)
    file_name = re.search(search, url).group(0)
    try:
        r = requests.get(url, headers = headers, allow_redirects=True)
        open(path+file_name, 'wb').write(r.content)
        print('Download Completed:', file_name)
    except Exception as error:
        print("Error in function: get_file()")
        print(error)

In [7]:
def get_links_ft(url='https://rebrickable.com/downloads/', file_type='.csv.gz'):
    '''
    Purpose: Parse a webpage for downloadable links based on file type
    Input: File Type and URL of file
    Default(s): File type of .csv.gz
    Output: Returns a list of links
    Notes: (blank)
    '''
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        a_ft = soup.find_all('a', href=re.compile(f'{file_type}'))
        for links in a_ft:
            print(f'URL Found for the file type of {file_type}:', links['href'])
        return a_ft
    except Exception as error:
        print("Error in function: get_links_ft()")
        print(error)

In [8]:
def download_files_rb(url='https://rebrickable.com/downloads/'):
    '''
    Purpose: Download Rebrickable compressed database files
    Input: URL
    Default(s): Rebrickable downloads page
    Output: (None)
    Notes: (Blank)
    '''
    try:
        print("Here are the links:")
        links = get_links_ft(url, '.csv.gz')
        print("Downloading:")
        for link in links:
            get_file(link['href'], '.csv.gz')
        print("Downloading Finished.")
        
    except Exception as error:
        print("Error in function: download_files_rb()")
        print(error)

In [9]:
def gunzip(file_in, file_out):
    '''
    Purpose: Unzip a Gzip a file
    Input: File path in, File path out
    Default(s): (None)
    Output: (None)
    Notes: (Blank)
    '''
    try:
        with gzip.open(file_in, 'rb') as f_in:
            with open(file_out, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
                print(f'Unzipped {file_in} to {file_out}')
    except Exception as error:
        print("Error in function: gunzip()")
        print(error)

In [10]:
def unzip_to_csv(path_in = './temp/', file_type_i = '.csv.gz'):
    '''
    Purpose: Unzip all gz files to to ,csv
    Input: Path in and file type
    Default(s): Path in and file type
    Output: (None)
    Notes: (Blank)
    '''
    try:
        lst = []
        file_type_o = '.csv'
        path_out = './data/csv/'
        search = re.compile(r'\w+'+file_type_i)
        for f_path_i in glob.glob(f'{path_in}*{file_type_i}'):
            f_path_o = path_out+re.sub(file_type_i, file_type_o,re.search(search, f_path_i).group(0))
            lst_t = [f_path_i, f_path_o]
            lst.append(lst_t)
        for file in lst:
            gunzip(file[0],file[1])
    except Exception as error:
        print("Error in function: unzip_to_csv()")
        print(error)

In [11]:
def write_json(file_path_out, list_dict):
    '''
    Purpose: Write a JSON file to a path
    Input: File_path out and a list of dictionaries
    Default(s): (None)
    Output: (None)
    Notes: (Blank)
    '''
    with open(file_path_out, 'w', encoding="utf8") as file_out:
        json.dump(list_dict, file_out)
    print(f'JSON file written to {file_path_out}')

In [12]:
def read_csv(file_path_in):
    '''
    Purpose: read a csv file and convert it to JSON
    Input: File path in
    Default(s): (None)
    Output: (None)
    Notes: (Blank)
    '''
    try:
        with open(file_path_in, 'r', encoding="utf8") as file_in:
            file = csv.DictReader(file_in, delimiter=',')
            data_s0 = []
            for row in file:
                data_s0.append(row)
            data = data_s0
        data_name = os.path.splitext(os.path.basename(file_path_in))[0]
        file_path_out = f'./data/json/{data_name}.json'
        write_json(file_path_out, data)
    except Exception as error:
        print("Error in function: read_csv()")
        print(error)

In [13]:
def convert_csv_json(path_in='./data/csv/'):
    '''
    Purpose: Convert a directory of CSVs to JSONs
    Input: File path in
    Default(s): File path in
    Output: (None)
    Notes: (Blank)
    '''
    file_list = os.listdir(path_in)
    for file in file_list:
        read_csv(f'{path_in}{file}')

In [14]:
def csvs_to_sql(path_in='./data/csv/'):
    '''
    Purpose: Convert a directory of CSVs to SQL tables
    Input: File path in
    Default(s): File path in
    Output: (None)
    Notes: (Blank)
    '''
    try:
        dbconn = sqlite3.connect(data_folder+'lego.db')
        lst = []
        search = re.compile(r'\w+\.csv')
        for f_path_i in glob.glob('./data/csv/*.csv'):
            name = re.sub('.csv', '', re.search(search, f_path_i).group(0))
            lst_t = [f_path_i, name]
            lst.append(lst_t)
            pandas.read_csv(f_path_i).to_sql(name,dbconn,if_exists='replace')
            print(f'Table {name} has been loaded from:\n{f_path_i}')
        close_DB_Resources(dbconn)
    except Exception as error:
        print("Error in function: csvs_to_sql()")
        print(error)   

In [15]:
def get_brickset_secrets(file_path ='./brickset.json'):
    '''
    Purpose: Use a JSON of secrets for use in the BrickSet API
    Input: File path in
    Default(s): File path in
    Output: apikey, username, password
    Notes: (Blank)
    '''
    apikey = ''
    username = ''
    password = ''
    
    try:
        with open(file_path, 'r') as keyfile:
            keydict = json.load(keyfile)
            apikey = keydict['apikey']
            username = keydict['username']
            password = keydict['password']
            return apikey, username, password
    except FileNotFoundError:
        print("Cound not find key file: ", file_path)
    except KeyError:
        print("Keyfile does not contain keyname")
    except Exception as error:
        print("Error in function: get_brickset_secrets()")
        print(error)

In [16]:
def check_brickset_api(apikey):
    '''
    Purpose: Check brickset apikey
    Input: apikey
    Default(s): (None)
    Output: Return the text of the API response
    Notes: (Blank)
    '''
    brickset_api_root = 'https://brickset.com/api/v3.asmx/'
    endpoint = f'checkKey?apiKey={apikey}'
    try:
        url = brickset_api_root + endpoint
        response = requests.get(url)
        if(int(response.status_code)==200):
            return response.text
        else:
            raise Exception("Non 200 status code")
    except Exception as error:
        print("Error in function: check_brickset_api()")
        print(error)

In [17]:
def get_brickset_hash(apikey,username,password):
    '''
    Purpose: Get userhash (login) for use with Brickset API 
    Input: Apikey, Username, Password
    Default(s): (None)
    Output: apikey, username, password
    Notes: (Blank)
    '''
    endpoint = f'login?apiKey={apikey}&username={username}&password={password}'
    try:
        url = brickset_api_root + endpoint
        response = requests.get(url)
        if(int(response.status_code)==200):
            r_json=response.json()
            return r_json['hash']
        else:
            raise Exception("Non 200 status code")
    except Exception as error:
        print("Error in function: get_brickset_hash()")
        print(error)

In [18]:
def get_brickset_sets(apikey,userHash,params={'owned':'1'}):
    '''
    Purpose: Retrive a dataset from the Brickset API based on parameters
    Input: Apikey, Userhash, Parameters
    Default(s): Parameters for owned sets
    Output: Json of results
    Notes: (Blank)
    '''
    endpoint = f'getSets?apiKey={apikey}&userHash={userHash}&params={params}'
    try:
        url = brickset_api_root + endpoint
        response = requests.get(url)
        if(int(response.status_code)==200):
            r_json=response.json()
            return r_json
        else:
            raise Exception("Non 200 status code")
    except Exception as error:
        print("Error in function: get_brickset_sets()")
        print(error)

In [19]:
def brickset_to_sql(dict_lst_dict, v_name, t_name):
    '''
    Purpose: Convert a Brickset Sets JSON to an SQL table 
    Input: JSON, Dataset Name, Table NAme 
    Default(s): (None)
    Output: (None)
    Notes: Link to one-liner: https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
    '''
    try:
        clean_list = ['collection', 'collections', 'LEGOCom', 'ageRange', 'dimensions', 'barcode', 'extendedData', 'US']
        drop_list = ['UK', 'CA', 'DE']
        drop_list.append(clean_list)
        dbconn = sqlite3.connect(data_folder+'lego.db')
        df = pandas.DataFrame(dict_lst_dict[v_name])
        df.drop('image', axis=1, inplace=True)
        for column in clean_list:
            expanded = df[column].apply(pandas.Series)
            df = pandas.concat([df, expanded], axis=1)
        for column in drop_list:
            df.drop(column, axis=1, inplace=True)
        df = df.loc[:,~df.columns.duplicated()] # From Stack Overflow
        df.to_sql(t_name,dbconn,if_exists='replace')
        print(f'Table {t_name} has been loaded from: {v_name}')
        close_DB_Resources(dbconn)
    except Exception as error:
        print("Error in function: brickset_to_sql()")
        print(error)

In [20]:
def get_table(table, db_path='./data/lego.db'):
    '''
    Purpose: Get a table from a SQL lite
    Input: JSON, Dataset Name, Table NAme 
    Default(s): (None)
    Output: (None)
    Notes: Link to one-liner: https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
    '''
    try:
        dbconn = sqlite3.connect(db_path)
        df = pandas.read_sql_query(f'SELECT * FROM {table}', dbconn)
        return df
        close_DB_Resources(dbconn)
    except Exception as error:
        print("Error in function: get_table()")
        print(error)

In [21]:
# Execution Layer

In [22]:
download_files_rb()

Here are the links:
URL Found for the file type of .csv.gz: https://cdn.rebrickable.com/media/downloads/themes.csv.gz?1639853278.890378
URL Found for the file type of .csv.gz: https://cdn.rebrickable.com/media/downloads/colors.csv.gz?1639853278.9903753
URL Found for the file type of .csv.gz: https://cdn.rebrickable.com/media/downloads/part_categories.csv.gz?1639853279.0863729
URL Found for the file type of .csv.gz: https://cdn.rebrickable.com/media/downloads/parts.csv.gz?1639853279.6503584
URL Found for the file type of .csv.gz: https://cdn.rebrickable.com/media/downloads/part_relationships.csv.gz?1639853284.742227
URL Found for the file type of .csv.gz: https://cdn.rebrickable.com/media/downloads/elements.csv.gz?1639853279.930351
URL Found for the file type of .csv.gz: https://cdn.rebrickable.com/media/downloads/sets.csv.gz?1639853280.5383353
URL Found for the file type of .csv.gz: https://cdn.rebrickable.com/media/downloads/minifigs.csv.gz?1639853280.8023286
URL Found for the file ty

In [23]:
unzip_to_csv()

Unzipped ./temp\colors.csv.gz to ./data/csv/colors.csv
Unzipped ./temp\elements.csv.gz to ./data/csv/elements.csv
Unzipped ./temp\inventories.csv.gz to ./data/csv/inventories.csv
Unzipped ./temp\inventory_minifigs.csv.gz to ./data/csv/inventory_minifigs.csv
Unzipped ./temp\inventory_parts.csv.gz to ./data/csv/inventory_parts.csv
Unzipped ./temp\inventory_sets.csv.gz to ./data/csv/inventory_sets.csv
Unzipped ./temp\minifigs.csv.gz to ./data/csv/minifigs.csv
Unzipped ./temp\parts.csv.gz to ./data/csv/parts.csv
Unzipped ./temp\part_categories.csv.gz to ./data/csv/part_categories.csv
Unzipped ./temp\part_relationships.csv.gz to ./data/csv/part_relationships.csv
Unzipped ./temp\sets.csv.gz to ./data/csv/sets.csv
Unzipped ./temp\themes.csv.gz to ./data/csv/themes.csv


In [24]:
convert_csv_json()

JSON file written to ./data/json/colors.json
JSON file written to ./data/json/elements.json
JSON file written to ./data/json/inventories.json
JSON file written to ./data/json/inventory_minifigs.json
JSON file written to ./data/json/inventory_parts.json
JSON file written to ./data/json/inventory_sets.json
JSON file written to ./data/json/minifigs.json
JSON file written to ./data/json/parts.json
JSON file written to ./data/json/part_categories.json
JSON file written to ./data/json/part_relationships.json
JSON file written to ./data/json/sets.json
JSON file written to ./data/json/themes.json


In [25]:
csvs_to_sql()

Table colors has been loaded from:
./data/csv\colors.csv
Table elements has been loaded from:
./data/csv\elements.csv
Table inventories has been loaded from:
./data/csv\inventories.csv
Table inventory_minifigs has been loaded from:
./data/csv\inventory_minifigs.csv
Table inventory_parts has been loaded from:
./data/csv\inventory_parts.csv
Table inventory_sets has been loaded from:
./data/csv\inventory_sets.csv
Table minifigs has been loaded from:
./data/csv\minifigs.csv
Table parts has been loaded from:
./data/csv\parts.csv
Table part_categories has been loaded from:
./data/csv\part_categories.csv
Table part_relationships has been loaded from:
./data/csv\part_relationships.csv
Table sets has been loaded from:
./data/csv\sets.csv
Table themes has been loaded from:
./data/csv\themes.csv

DB resources were closed successfully.


In [26]:
apikey, username, password = get_brickset_secrets()

In [27]:
check_brickset_api(apikey)

'{"status":"success"}'

In [28]:
userhash = get_brickset_hash(apikey, username, password)

In [29]:
owned_sets = get_brickset_sets(apikey, userhash,{'owned':'1'})

In [30]:
owned_sets

{'status': 'success',
 'matches': 207,
 'sets': [{'setID': 2813,
   'number': '5571',
   'numberVariant': 1,
   'name': 'Giant Truck',
   'year': 1996,
   'theme': 'Model Team',
   'themeGroup': 'Model making',
   'category': 'Normal',
   'released': True,
   'pieces': 1757,
   'image': {'thumbnailURL': 'https://images.brickset.com/sets/small/5571-1.jpg',
    'imageURL': 'https://images.brickset.com/sets/images/5571-1.jpg'},
   'bricksetURL': 'https://brickset.com/sets/5571-1',
   'collection': {'owned': True,
    'wanted': False,
    'qtyOwned': 1,
    'rating': 0,
    'notes': ''},
   'collections': {'ownedBy': 1289, 'wantedBy': 1872},
   'LEGOCom': {'US': {'retailPrice': 139.0}, 'UK': {}, 'CA': {}, 'DE': {}},
   'rating': 4.7,
   'reviewCount': 7,
   'packagingType': 'Box',
   'availability': 'Retail',
   'instructionsCount': 0,
   'additionalImageCount': 0,
   'ageRange': {},
   'dimensions': {},
   'barcode': {'EAN': '5702010955718'},
   'extendedData': {},
   'lastUpdated': '2021

In [31]:
wanted_sets = get_brickset_sets(apikey, userhash,{'wanted':'1'})

In [32]:
wanted_sets

{'status': 'success',
 'matches': 281,
 'sets': [{'setID': 6922,
   'number': '5974',
   'numberVariant': 1,
   'name': 'Galactic Enforcer',
   'year': 2009,
   'theme': 'Space',
   'themeGroup': 'Action/Adventure',
   'subtheme': 'Space Police 3',
   'category': 'Normal',
   'released': True,
   'pieces': 825,
   'minifigs': 7,
   'image': {'thumbnailURL': 'https://images.brickset.com/sets/small/5974-1.jpg',
    'imageURL': 'https://images.brickset.com/sets/images/5974-1.jpg'},
   'bricksetURL': 'https://brickset.com/sets/5974-1',
   'collection': {'owned': False,
    'wanted': True,
    'qtyOwned': 0,
    'rating': 0,
    'notes': ''},
   'collections': {'ownedBy': 2724, 'wantedBy': 1980},
   'LEGOCom': {'US': {'retailPrice': 99.99,
     'dateFirstAvailable': '2009-06-28T00:00:00Z',
     'dateLastAvailable': '2010-10-22T00:00:00Z'},
    'UK': {'retailPrice': 64.99,
     'dateFirstAvailable': '2009-06-04T00:00:00Z',
     'dateLastAvailable': '2010-11-25T00:00:00Z'},
    'CA': {'retail

In [33]:
brickset_to_sql(owned_sets,'sets','owned_sets')

Table owned_sets has been loaded from: sets

DB resources were closed successfully.


In [34]:
brickset_to_sql(wanted_sets,'sets','wanted_sets')

Table wanted_sets has been loaded from: sets

DB resources were closed successfully.


In [35]:
get_table('wanted_sets')

,index,setID,number,numberVariant,name,year,theme,themeGroup,subtheme,category,...,max,height,width,depth,weight,EAN,UPC,retailPrice,dateFirstAvailable,dateLastAvailable
0,0,6922,5974,1,Galactic Enforcer,2009,Space,Action/Adventure,Space Police 3,Normal,...,14.0,37.8000,58.2000,8.7100,1.680,5702014533066,673419111614,99.99,2009-06-28T00:00:00Z,2010-10-22T00:00:00Z
1,1,7595,5984,1,Lunar Limo,2010,Space,Action/Adventure,Space Police 3,Normal,...,14.0,28.1900,48.0100,6.1000,0.770,5702014602366,673419129954,39.99,2010-05-29T00:00:00Z,2010-12-25T00:00:00Z
2,2,8577,7066,1,Earth Defense HQ,2011,Space,Action/Adventure,Alien Conquest,Normal,...,14.0,37.8000,58.1900,8.7100,1.660,5702014736856,673419145800,89.99,2011-05-13T00:00:00Z,2012-04-04T00:00:00Z
3,3,6423,7644,1,MX-81 Hypersonic Operations Aircraft,2008,Space,Action/Adventure,Mars Mission,Normal,...,NaN,38.4000,57.6000,7.2000,1.890,None,673419101486,79.99,2008-05-26T00:00:00Z,2009-03-10T00:00:00Z
4,4,7610,7937,1,Train Station,2010,City,Modern day,Trains,Normal,...,12.0,28.2000,48.0000,6.1000,NaN,5702014602588,673410130317,49.99,2010-06-07T00:00:00Z,2013-12-26T00:00:00Z
5,5,5793,7997,1,Train Station,2007,City,Modern day,Trains,Normal,...,NaN,28.8000,38.4000,9.6000,NaN,5702014499034,673419090995,44.99,2007-10-24T00:00:00Z,2010-01-13T00:00:00Z
6,6,4240,8386,1,Ferrari F1 Racer 1:10,2004,Racers,Racing,Ferrari,Normal,...,NaN,38.4000,57.6000,7.2000,1.400,None,673419036337,70.00,None,None
7,7,4300,8485,1,Control Centre II,1995,Technic,Technical,None,Normal,...,NaN,NaN,NaN,NaN,NaN,None,None,219.00,None,None
8,8,31372,9287,1,Bonus Lego Basic Town,1996,Education,Educational,Town,Normal,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None
9,9,31665,9452,1,Giant LEGO topic set,1991,Dacta,Educational,None,Normal,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None


In [36]:
get_table('owned_sets')

,index,setID,number,numberVariant,name,year,theme,themeGroup,category,released,...,max,height,width,depth,weight,EAN,UPC,retailPrice,dateFirstAvailable,dateLastAvailable
0,0,2813,5571,1,Giant Truck,1996,Model Team,Model making,Normal,1,...,NaN,NaN,NaN,NaN,NaN,5702010955718,None,139.00,None,None
1,1,2818,5591,1,Mach II Red Bird Rig,1994,Model Team,Model making,Normal,1,...,NaN,NaN,NaN,NaN,NaN,5702010955916,None,110.00,None,None
2,2,3743,6973,1,Deep Freeze Defender,1993,Space,Action/Adventure,Normal,1,...,NaN,NaN,NaN,NaN,NaN,None,042884069735,45.00,None,None
3,3,3752,6984,1,Galactic Mediator,1992,Space,Action/Adventure,Normal,1,...,NaN,NaN,NaN,NaN,NaN,None,042884069841,60.00,None,None
4,4,4283,8460,1,Pneumatic Crane Truck,1995,Technic,Technical,Normal,1,...,NaN,NaN,NaN,NaN,NaN,None,None,99.00,None,None
5,5,4437,8714,1,The LEGO Technic Guys,1993,Technic,Technical,Normal,1,...,NaN,NaN,NaN,NaN,NaN,None,None,9.00,None,None
6,6,4508,8865,1,Test Car,1988,Technic,Technical,Normal,1,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None
7,7,4511,8880,1,Super Car,1994,Technic,Technical,Normal,1,...,NaN,NaN,NaN,NaN,NaN,5702010988808,None,130.00,None,None
8,8,8049,10214,1,Tower Bridge,2010,Creator Expert,Model making,Normal,1,...,NaN,48.0000,58.2000,12.7000,4.350000,5702015013208,673419128971,239.99,2010-10-03T00:00:00Z,2019-09-29T00:00:00Z
9,9,8704,10218,1,Pet Shop,2011,Creator Expert,Model making,Normal,1,...,NaN,48.0000,58.2000,9.1000,NaN,673419145350,673419145350,149.99,2011-05-12T00:00:00Z,2016-11-26T00:00:00Z


In [37]:
get_table('owned_ppp_n','C:\\Users\\Rashan\\Documents\\CCAC\\DAT-204\\21fa_r_for_analytics\\Final_Project\\data\\lego.db')

,name,ppp,n
0,48x48 Grey Baseplate,14.990000,1
1,32x32 Green Baseplate,7.990000,1
2,Blue Baseplate,7.990000,1
3,The LEGO Technic Guys,1.000000,1
4,Galactic Mediator,0.147783,1
5,Winter Holiday Train,0.136226,1
6,Pneumatic Crane Truck,0.116471,1
7,James Bond Aston Martin DB5,0.116271,1
8,The Eiffel Tower,0.109003,1
9,Deep Freeze Defender,0.107914,1
